In [3]:
from __future__ import absolute_import, division, print_function

import numpy as np
import time

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter

import json

from utils import *
from kitti_utils import *
from layers import *

import datasets
import networks
from IPython import embed

In [2]:
datasets_dict = {"kitti": datasets.KITTIRAWDataset,
                 "kitti_odom": datasets.KITTIOdomDataset}
dataset = datasets_dict["kitti"]
fpath = os.path.join("splits", "eigen_zhou", "{}_files.txt")
train_filenames = readlines(fpath.format("train"))
val_filenames = readlines(fpath.format("val"))

In [3]:
train_dataset = dataset(
    "kitti_data", train_filenames, 192, 640,
    [0, -1, 1], 4, is_train=True, img_ext='.png')

train_loader = DataLoader(
    train_dataset, 4, True,
    num_workers=4, pin_memory=True, drop_last=True)


c:\Users\wongyun\miniconda3\envs\torchstudy\lib\site-packages\torchvision\transforms\transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [4]:
for batch in train_loader:
    for key, ipt in batch.items():
        print("key:", key)
        print("value.shape: ", ipt.shape)
    break

key: ('K', 0)
value.shape:  torch.Size([4, 4, 4])
key: ('inv_K', 0)
value.shape:  torch.Size([4, 4, 4])
key: ('K', 1)
value.shape:  torch.Size([4, 4, 4])
key: ('inv_K', 1)
value.shape:  torch.Size([4, 4, 4])
key: ('K', 2)
value.shape:  torch.Size([4, 4, 4])
key: ('inv_K', 2)
value.shape:  torch.Size([4, 4, 4])
key: ('K', 3)
value.shape:  torch.Size([4, 4, 4])
key: ('inv_K', 3)
value.shape:  torch.Size([4, 4, 4])
key: ('color', 0, 0)
value.shape:  torch.Size([4, 3, 192, 640])
key: ('color', 0, 1)
value.shape:  torch.Size([4, 3, 96, 320])
key: ('color', 0, 2)
value.shape:  torch.Size([4, 3, 48, 160])
key: ('color', 0, 3)
value.shape:  torch.Size([4, 3, 24, 80])
key: ('color', -1, 0)
value.shape:  torch.Size([4, 3, 192, 640])
key: ('color', -1, 1)
value.shape:  torch.Size([4, 3, 96, 320])
key: ('color', -1, 2)
value.shape:  torch.Size([4, 3, 48, 160])
key: ('color', -1, 3)
value.shape:  torch.Size([4, 3, 24, 80])
key: ('color', 1, 0)
value.shape:  torch.Size([4, 3, 192, 640])
key: ('color

In [5]:
for batch in train_loader:
    print(batch[("color_aug", 0, 0)].shape)
    break

torch.Size([4, 3, 192, 640])


In [1]:
from swin_transformer_v2 import SwinTransformerV2

from swin_transformer_v2 import swin_transformer_v2_t, swin_transformer_v2_s, swin_transformer_v2_b, \
    swin_transformer_v2_l, swin_transformer_v2_h, swin_transformer_v2_g

# SwinV2-T
swin_transformer: SwinTransformerV2 = swin_transformer_v2_t(in_channels=3,
                                                            window_size=8,
                                                            input_resolution=(256, 256),
                                                            sequential_self_attention=False,
                                                            use_checkpoint=False)

swin_transformer.update_resolution(new_window_size=2, new_input_resolution=(192, 640))

c:\Users\wongyun\miniconda3\envs\torchstudy\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [49]:
# 32로 나눈 다음에 window size로 또 나눌 수 있어야함 
for batch in train_loader:
    depth_data = batch[("color_aug", 0, 0)]
    output=swin_transformer(depth_data)[3]
    print(output.shape)
    break

torch.Size([4, 768, 6, 20])


In [2]:
import torch
dummy=torch.randn(4,3,510,256)
result=swin_transformer(dummy)
print(result[0].shape)
print(result[1].shape)
print(result[2].shape)
print(result[3].shape)

RuntimeError: shape '[3, 63, 32, 96, 2, 2]' is invalid for input of size 3096576

In [26]:
swin_transformer.update_resolution(new_window_size=8, new_input_resolution=(512, 512))

dummy=torch.randn(4,3,512,512)
result=swin_transformer(dummy)
print(result[0].shape)
print(result[1].shape)
print(result[2].shape)
print(result[3].shape)

torch.Size([4, 96, 128, 128])
torch.Size([4, 192, 64, 64])
torch.Size([4, 384, 32, 32])
torch.Size([4, 768, 16, 16])


In [31]:
swin_transformer.update_resolution(new_window_size=2, new_input_resolution=(320, 320))

dummy=torch.randn(4,3,320,320)
result=swin_transformer(dummy)
print(result[0].shape)
print(result[1].shape)
print(result[2].shape)
print(result[3].shape)

torch.Size([4, 96, 80, 80])
torch.Size([4, 192, 40, 40])
torch.Size([4, 384, 20, 20])
torch.Size([4, 768, 10, 10])


In [47]:
swin_transformer.update_resolution(new_window_size=2, new_input_resolution=(192, 640))

dummy=torch.randn(4,3, 192, 640)
result=swin_transformer(dummy)
print(result[0].shape)
print(result[1].shape)
print(result[2].shape)
print(result[3].shape)

torch.Size([4, 96, 48, 160])
torch.Size([4, 192, 24, 80])
torch.Size([4, 384, 12, 40])
torch.Size([4, 768, 6, 20])


In [9]:
import torch

dummy=torch.randn(4, 3, 256, 128)
logits=swin_transformer(dummy)

print(logits[0].shape)
print(logits[1].shape)
print(logits[2].shape)
print(logits[3].shape)

RuntimeError: The size of tensor a (32) must match the size of tensor b (16) at non-singleton dimension 3

In [3]:
print(logits[0].shape)
print(logits[1].shape)
print(logits[2].shape)
print(logits[3].shape)


torch.Size([4, 96, 64, 64])
torch.Size([4, 192, 32, 32])
torch.Size([4, 384, 16, 16])
torch.Size([4, 768, 8, 8])


In [4]:
from swin_transformer_v2 import SwinTransformerV2

from swin_transformer_v2 import swin_transformer_v2_t, swin_transformer_v2_s, swin_transformer_v2_b, \
    swin_transformer_v2_l, swin_transformer_v2_h, swin_transformer_v2_g

# SwinV2-T
swin_transformer: SwinTransformerV2 = swin_transformer_v2_t(in_channels=3,
                                                            window_size=(6, 20),
                                                            input_resolution=(192, 640),
                                                            sequential_self_attention=False,
                                                            use_checkpoint=False)

In [5]:
import torch

dummy=torch.randn(4, 3, 192, 640)
result=swin_transformer(dummy)
print(result[0].shape)
print(result[1].shape)
print(result[2].shape)
print(result[3].shape)

torch.Size([4, 96, 48, 160])
torch.Size([4, 192, 24, 80])
torch.Size([4, 384, 12, 40])
torch.Size([4, 768, 6, 20])


In [3]:
for i in range(4, -1, -1):
    print(i)

4
3
2
1
0


In [5]:
import torch.nn.functional as F
import torch

x=torch.randn(4, 768, 6, 20)
x=F.interpolate(x, scale_factor=2, mode="nearest")

print(x.shape)

torch.Size([4, 768, 12, 40])


In [1]:
import torch
import timm

In [2]:
model=timm.create_model('resnet34', features_only=True)
dummy=torch.randn(8,3,224,224)

for i in model(dummy):
    print(i.shape)

torch.Size([8, 64, 112, 112])
torch.Size([8, 64, 56, 56])
torch.Size([8, 128, 28, 28])
torch.Size([8, 256, 14, 14])
torch.Size([8, 512, 7, 7])


In [8]:
model=timm.create_model('swinv2_tiny_window8_256', pretrained=True, features_only=True)
dummy=torch.randn(8,3,256,256)

model(dummy)

AttributeError: 'SwinTransformerV2' object has no attribute 'feature_info'

In [10]:
model=timm.create_model('swin_v2_cr_tiny_ns_224', pretrained=True)

In [9]:
model=timm.create_model('vgg19', features_only=True)

In [11]:
for i in model(dummy):
    print(i.shape)

torch.Size([8, 64, 256, 256])
torch.Size([8, 128, 128, 128])
torch.Size([8, 256, 64, 64])
torch.Size([8, 512, 32, 32])
torch.Size([8, 512, 16, 16])
torch.Size([8, 512, 8, 8])


In [14]:
print(model.forward_features(dummy).shape)
print(model.forward_features(dummy).shape)

torch.Size([8, 768, 7, 7])
torch.Size([8, 768, 7, 7])


In [13]:
train_dir='tmp/mono_model/models/pretrained/swin_v2_cr_tiny_ns_224-ba8166c6.pth'
pretrained_dir='tmp/mono_model/models/weights_19/pose_encoder.pth'

In [14]:
import torch
train_model=torch.load(train_dir)
pretrained_model=torch.load(pretrained_dir)

In [18]:
pretrained_model.keys()

odict_keys(['encoder.conv1.weight', 'encoder.bn1.weight', 'encoder.bn1.bias', 'encoder.bn1.running_mean', 'encoder.bn1.running_var', 'encoder.bn1.num_batches_tracked', 'encoder.layer1.0.conv1.weight', 'encoder.layer1.0.bn1.weight', 'encoder.layer1.0.bn1.bias', 'encoder.layer1.0.bn1.running_mean', 'encoder.layer1.0.bn1.running_var', 'encoder.layer1.0.bn1.num_batches_tracked', 'encoder.layer1.0.conv2.weight', 'encoder.layer1.0.bn2.weight', 'encoder.layer1.0.bn2.bias', 'encoder.layer1.0.bn2.running_mean', 'encoder.layer1.0.bn2.running_var', 'encoder.layer1.0.bn2.num_batches_tracked', 'encoder.layer1.1.conv1.weight', 'encoder.layer1.1.bn1.weight', 'encoder.layer1.1.bn1.bias', 'encoder.layer1.1.bn1.running_mean', 'encoder.layer1.1.bn1.running_var', 'encoder.layer1.1.bn1.num_batches_tracked', 'encoder.layer1.1.conv2.weight', 'encoder.layer1.1.bn2.weight', 'encoder.layer1.1.bn2.bias', 'encoder.layer1.1.bn2.running_mean', 'encoder.layer1.1.bn2.running_var', 'encoder.layer1.1.bn2.num_batches_tr

In [19]:
train_model.keys()

dict_keys(['patch_embed.proj.weight', 'patch_embed.proj.bias', 'patch_embed.norm.weight', 'patch_embed.norm.bias', 'stages.0.blocks.0.attn.tau', 'stages.0.blocks.0.attn.qkv.weight', 'stages.0.blocks.0.attn.qkv.bias', 'stages.0.blocks.0.attn.proj.weight', 'stages.0.blocks.0.attn.proj.bias', 'stages.0.blocks.0.attn.meta_mlp.fc1.weight', 'stages.0.blocks.0.attn.meta_mlp.fc1.bias', 'stages.0.blocks.0.attn.meta_mlp.fc2.weight', 'stages.0.blocks.0.attn.meta_mlp.fc2.bias', 'stages.0.blocks.0.norm1.weight', 'stages.0.blocks.0.norm1.bias', 'stages.0.blocks.0.mlp.fc1.weight', 'stages.0.blocks.0.mlp.fc1.bias', 'stages.0.blocks.0.mlp.fc2.weight', 'stages.0.blocks.0.mlp.fc2.bias', 'stages.0.blocks.0.norm2.weight', 'stages.0.blocks.0.norm2.bias', 'stages.0.blocks.1.attn.tau', 'stages.0.blocks.1.attn.qkv.weight', 'stages.0.blocks.1.attn.qkv.bias', 'stages.0.blocks.1.attn.proj.weight', 'stages.0.blocks.1.attn.proj.bias', 'stages.0.blocks.1.attn.meta_mlp.fc1.weight', 'stages.0.blocks.1.attn.meta_mlp.fc

In [3]:
import torch
import torch.nn as nn

N, C, H, W = 4, 8, 8, 768
input = torch.randn(N, C, H, W)
norm=nn.LayerNorm(768)

result=norm(input)

In [9]:
pool=torch.randn(4, 768, 1)
pool_new=torch.flatten(pool, 1)
pool_new.shape

torch.Size([4, 768])

In [13]:
import torch.nn as nn
import torch

x=torch.randn(4, 32, 128)

layers = nn.Sequential(
    nn.Linear(96*2, 512),
    nn.ReLU(),
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Linear(512, 512)
)


In [15]:
def x():
    a=3
    print(a)
    return a

k=[]
k.append(x())

3
